<a href="https://colab.research.google.com/github/Shankars57/Deep_Learning/blob/main/IMG_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# Load the cats_vs_dogs dataset
dataset_name = "cats_vs_dogs"
dataset, info = tfds.load(dataset_name, as_supervised=True, with_info=True)

train_data = dataset['train'].take(5000)
val_data = dataset['train'].skip(20000).take(1000)  # Next 5000 for validation

# Preprocess function
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image, label

# Apply preprocessing and batching
train_data = train_data.map(preprocess).batch(128).shuffle(1000)
val_data = val_data.map(preprocess).batch(128)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), padding="same"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, validation_data=val_data, epochs=3)

# Evaluate the model
loss, acc = model.evaluate(val_data)
print(f"\nValidation Accuracy: {acc * 100:.2f}%")

# Function to display a predicted image
def show_prediction():
    image, label = next(iter(val_data))  # Get a batch from the validation data
    img = image[0].numpy()  # Convert tensor to numpy array
    true_label = label[0].numpy()

    # Make prediction
    prediction = model.predict(tf.expand_dims(image[0], axis=0))  # Add batch dimension
    predicted_label = "Dog" if prediction[0][0] > 0.5 else "Cat"

    # Display the image with predicted and actual labels
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_label}, Actual: {'Dog' if true_label else 'Cat'}")
    plt.axis("off")
    plt.show()

# Show a random predicted image
show_prediction()
